In [5]:
!pip install medmnist giotto-tda scikit-learn matplotlib seaborn gudhi

In [8]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import medmnist
import cv2
import skimage
import shap
import pandas as pd
import xgboost as xgb

import gudhi as gd
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import classification_report, roc_auc_score, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split

from gtda.homology import CubicalPersistence
from gtda.diagrams import PersistenceImage
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_diagram
from medmnist import BreastMNIST

from skimage import feature
import skimage.transform as tf

# Set random seed
#np.random.seed(42)

In [14]:
# Load BreastMNIST dataset (2D grayscale breast ultrasound images)

data = BreastMNIST(split='train', download=True, size=64)
x_train, y_train = data.imgs, data.labels


# 80:20 split
X_train, X_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print("Train:", X_train.shape, "Test:", X_test.shape, "Val:", x_val.shape)

Train: (436, 64, 64) Test: (110, 64, 64) Val: (78, 64, 64)
